install necessary packages

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandasql as ps
import re
import matplotlib.pyplot as plt


reading in the two Jane Austen books from project gutenberg: "Sense and Sensibility" and "Pride and Prejudice"

In [50]:
url_sense = "https://www.gutenberg.org/files/21839/21839-0.txt"
url_pride = "https://www.gutenberg.org/files/1342/1342-0.txt"

response_sense = requests.get(url_sense)
response_pride = requests.get(url_pride)

if response_sense.status_code == 200 and response_pride.status_code == 200:
    sense_data = response_sense.text
    pride_data = response_pride.text

Next I'll merge these to dat frames with a join

In [51]:
# Data Cleaning and Merge
sense_words = sense_data.split()
pride_words = pride_data.split()

sense_df = pd.DataFrame({"Words": sense_words})
pride_df = pd.DataFrame({"Words": pride_words})

merged_df = pd.merge(sense_df, pride_df, on="Words", how="inner")

Next I will clean the data some by breaking some of this text apart by sentences. I'll look for normal sentence breaks like "!,?,." to make sense of where each sentence starts and stops.

I'll do this for both books

In [52]:
 # Data Cleaning and Splitting into Sentences
sentences_sense = re.split(r'\. |\? |\! ', sense_data)  # Splitting based on sentence endings
sentences_pride = re.split(r'\. |\? |\! ', pride_data) 

Next, I'd like to calcultate the word count in each of these sentences.

In [53]:
# Calculate word count for each sentence
sentence_word_counts_sense = [len(sentence.split()) for sentence in sentences_sense]
sentence_word_counts_pride = [len(sentence.split()) for sentence in sentences_pride]

Next I'd like to make columns for this sentence data in the data frames I've already made. 

In [54]:
  # Add the sentence data and cumulative word counts to the sense_data DataFrame
sense_data_df = pd.DataFrame({
        "Sentence": sentences_sense,
    
    })

 # Add the sentence data and cumulative word counts to the pride_data DataFrame
pride_data_df = pd.DataFrame({
        "Sentence": sentences_pride,
      
    })

In [55]:


# Calculate word count for each sentence
#sentence_word_counts_sense = [len(sentence.split()) for sentence in sentences_sense]
#sentence_word_counts_pride = [len(sentence.split()) for sentence in sentences_pride]

# Calculate cumulative word count for sentences
#cumulative_word_counts_sense = [sum(sentence_word_counts_sense[:i+1]) for i in range(len(sentence_word_counts_sense))]
#cumulative_word_counts_pride = [sum(sentence_word_counts_pride[:i+1]) for i in range(len(sentence_word_counts_pride))]




In [56]:
# Data Cleaning and Merge
sense_words = sense_data.split()
pride_words = pride_data.split()

sense_df = pd.DataFrame({"Words": sense_words})
pride_df = pd.DataFrame({"Words": pride_words})

merged_df = pd.merge(sense_df, pride_df, on="Words", how="inner")


In [57]:
merged_df.head()

,Words
0,ï»¿The
1,Project
2,Project
3,Project
4,Project


In [58]:
word_count_comparison = merged_df["Words"].value_counts()

sentence_length_sense = sense_data.split(".")
sentence_length_pride = pride_data.split(".")

avg_sentence_length_sense = sum(len(sentence.split()) for sentence in sentence_length_sense) / len(sentence_length_sense)
avg_sentence_length_pride = sum(len(sentence.split()) for sentence in sentence_length_pride) / len(sentence_length_pride)


In [59]:
print("Word Count Comparison:")
print(word_count_comparison)

print("\nAverage Sentence Length (Sense and Sensibility):")
print(avg_sentence_length_sense)

print("\nAverage Sentence Length (Pride and Prejudice):")
print(avg_sentence_length_pride)

Word Count Comparison:
Words
the           18743913
to            17698500
of            14683896
and           11613239
a              4242079
                ...   
deceived;            1
miserable;           1
forwarded            1
gone;                1
eBooks.              1
Name: count, Length: 6910, dtype: int64

Average Sentence Length (Sense and Sensibility):
23.867677725118483

Average Sentence Length (Pride and Prejudice):
19.716469354113592


In [60]:
# Line Chart
#plt.subplot(1, 3, 2)
#plt.plot([0, 1], [avg_sentence_length_sense, avg_sentence_length_pride], marker="o")
#plt.title("Average Sentence Length")
#plt.xticks([0, 1], ["Sense and Sensibility", "Pride and Prejudice"])
#plt.xlabel("Novel")
#plt.ylabel("Average Length")


In [61]:
query = "SELECT Words, COUNT(*) as Count FROM merged_df GROUP BY Words ORDER BY Count DESC LIMIT 5"
result = ps.sqldf(query) 

# Pie Chart
plt.subplot(1, 3, 3)
plt.pie(result["Count"], labels=result["Words"], autopct="%1.1f%%")
plt.title("Most Common Words")
plt.axis("equal")

plt.tight_layout()
plt.show()

In [ ]:
# Data Cleaning and Splitting into Chapters
chapters = re.split(r'\bChapter \d+\b', pride_data)[1:]  # Splitting based on chapter headings
chapter_numbers = [f"Chapter {i}" for i in range(1, len(chapters) + 1)]

# Calculate word count for each chapter
word_counts = [len(chapter.split()) for chapter in chapters]

# Create a DataFrame to store the chapter data
chapter_df = pd.DataFrame({
        "Chapter": chapter_numbers,
        "Content": chapters,
        "Word_Count": word_counts
    })

 # Display the DataFrame
print(chapter_df)


Empty DataFrame
Columns: [Chapter, Content, Word_Count]
Index: []
